## relevance traininng

In [1]:
import random
import math
from torch.utils.data import Dataset
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import time
import json
import warnings
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.metrics import confusion_matrix
from collections import defaultdict
import numpy as np
from rank_bm25 import BM25Okapi
import spacy
import tensorflow as tf
import string
warnings.simplefilter(action='ignore', category=FutureWarning)

TRAIN_PATH = "./data/train-claims.json"
EVI_PATH = "./data/evidence.json"
NER_PATH = "./data/spacy_ner_tot.json"
PAD = "[PAD]"
CLS = '[CLS]'
SEP = '[SEP]'
PAD_LEN = 50
RANDOM_SEED = 42
SUPPORTS = "SUPPORTS"
REFUTES = "REFUTES"
NOT_ENOUGH_INFO = "NOT_ENOUGH_INFO"
DISPUTED = "DISPUTED"
RELATED = 1
NOT_RELATED = 0


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
porter_stemmer = PorterStemmer()
spacy_nlp = spacy.load("en_core_web_sm")
puncu_remove = str.maketrans('', '', string.punctuation)
with open(NER_PATH, "r") as cur_file:
    ner_dict = json.loads(cur_file.read())


2023-05-14 21:35:31.381751: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-14 21:35:31.549986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 21:35:32.452171: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-14 21:35:33.644808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-14 21:35:33.688

### Preprocessing

In [2]:

## __________ Setup Bert __________
claim_id_df = pd.read_json(TRAIN_PATH, orient="index")
evi_id_df = pd.read_json(EVI_PATH, orient="index").rename({0: "evi_text"}, axis=1)
claim_df = claim_id_df.reset_index(names="claim")
evi_df = evi_id_df.reset_index(names="evidence")




match BM25

In [3]:
def preprocess(input):
    # tokenize
    tokens = word_tokenize(input.lower().translate(puncu_remove))
    # remove stop words, lemmatize
    tokens = [lemmatizer.lemmatize(i) for i in tokens if i not in stop_words]
    
    return tokens

def get_n_bm25(bm25, text, n):
    # Retrive and select top n bm25 score
    token = preprocess(text)
    scores = bm25.get_scores(token)
    scores = np.array([i for i in enumerate(scores)])
    top_n_indices = [[int(i[0]), float(i[1])] for i in scores[scores[:, 1].argsort()][-n:]]
    print([i[0] for i in top_n_indices])
    return [i[0] for i in top_n_indices]

In [4]:
## 43 min
max_evi = 10

with tf.device('/GPU:0'):
    document = evi_df["evi_text"].values
    document = [preprocess(i) for i in document]
    bm25 = BM25Okapi(document)


with tf.device('/GPU:0'):
    # pred_df = claim_df[["claim", "claim_text"]]
    pred_df = claim_df[["claim", "claim_text", "claim_label", "evidences"]]
    pred_df["pred_evidence"] = pred_df.apply(lambda x: [evi_df.loc[i]["evidence"] for i in get_n_bm25(bm25, x["claim_text"], max_evi)], axis=1)
    print(pred_df)



2023-05-14 21:35:38.817825: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-14 21:35:38.818335: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-14 21:35:38.818793: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-14 21:35:47.650335: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-14 21:35:47.651506: I tensorflow/compile

[835824, 55991, 403020, 12425, 32852, 668884, 55589, 256886, 364767, 66273]
[590642, 876348, 343804, 211172, 430393, 1081418, 606276, 625160, 722506, 280331]
[380159, 938928, 354602, 1208158, 984887, 410946, 420577, 32728, 667390, 84969]
[352655, 1007867, 782433, 268745, 796796, 543549, 352729, 954557, 230669, 929977]
[917477, 646839, 625160, 348639, 1005096, 898633, 951865, 594052, 343576, 110980]
[833614, 256985, 632818, 429049, 576055, 996584, 1208236, 551233, 694262, 169703]
[160755, 534204, 1049197, 700842, 602109, 994854, 253111, 923298, 974673, 337932]
[449255, 514415, 62260, 491035, 1072749, 1174308, 865235, 118491, 1183793, 1189711]
[863309, 990887, 1063308, 1188266, 722506, 253691, 729613, 757821, 646402, 268048]
[296134, 131765, 714864, 668884, 805335, 1140805, 962456, 66273, 364767, 100018]
[527125, 611590, 292357, 103130, 1078581, 339274, 1057600, 789934, 853312, 172698]
[529564, 118277, 754909, 879956, 324930, 841806, 539241, 755608, 592588, 2338]
[618749, 915569, 246977,

Match NER

In [5]:
max_evi = 0
ner_hurdle = 500



ner_df = pd.DataFrame(columns=["claim", "evidence"])

# match NER on between claim and evidence
for cur_id, cur_text in claim_df[["claim", "claim_text"]].values[:100]:
    doc = spacy_nlp(cur_text)
    for ent in doc.ents:
        if ent.label_ in ner_dict and ent.text.lower().translate(puncu_remove) in ner_dict[ent.label_]:
            if len(ner_dict[ent.label_][ent.text.lower().translate(puncu_remove)]) > ner_hurdle: continue
            for evi in ner_dict[ent.label_][ent.text.lower().translate(puncu_remove)]:
                ner_df = ner_df.append({"claim": cur_id, "evidence":evi}, ignore_index=True)
            

In [6]:
# reformate
rel_df = pd.DataFrame(columns=["claim", "evidence"])
for claim, evidence in pred_df[["claim", "pred_evidence"]].values:
    for evi in evidence:
        rel_df = rel_df.append({"claim": claim, "evidence":evi}, ignore_index=True)
rel_df = pd.concat([rel_df, ner_df]).reset_index(drop=True).rename({"claim": "claim_id", "evidence": "evidence_id"}, axis=1)
print(rel_df.head(1))



     claim_id      evidence_id
0  claim-1937  evidence-835824


In [7]:
# sample fraction of training data
sample_frac = 0.1
train_df = rel_df[["claim_id", "evidence_id"]].sample(frac=sample_frac).reset_index(drop=True)
train_df["label"] = NOT_RELATED
train_df = train_df.set_index(["claim_id", "evidence_id"])
append_df = pd.DataFrame(columns=["claim_id", "evidence_id", "label"])

## get claim and evidence text for training
for claim_id, claim, label, evidence in claim_df.values:
    for evi in evidence:
        if (claim_id, evi) in train_df:
            train_df.loc[(claim_id, evi)]["label"] = RELATED
        else:
            append_df = append_df.append({"claim_id": claim_id,"evidence_id": evi, "label": RELATED}, ignore_index=True)

train_df = pd.concat([train_df.reset_index(), append_df]).reset_index(drop=True)
train_df['claim'] = train_df.apply(lambda x: claim_id_df.loc[x["claim_id"]]["claim_text"], axis=1)
train_df['evidence'] = train_df.apply(lambda x: evi_id_df.loc[x["evidence_id"]]["evi_text"], axis=1)
print(train_df)
print(train_df.shape)


        claim_id       evidence_id label  \
0      claim-499   evidence-977199     0   
1     claim-1792   evidence-512030     0   
2     claim-2283  evidence-1110598     0   
3      claim-428   evidence-600968     0   
4     claim-3104    evidence-66482     0   
...          ...               ...   ...   
6101   claim-502   evidence-583187     1   
6102  claim-3093   evidence-971105     1   
6103  claim-3093   evidence-457769     1   
6104  claim-3093   evidence-298971     1   
6105  claim-3093   evidence-883158     1   

                                                  claim  \
0     The monetary cost of damages has increased dra...   
1     Greenland's ice loss is accelerating & will ad...   
2     In fact, the global melt rate has been acceler...   
3           Brooks added that Antarctic ice is growing.   
4     the satellite sensors show less warming in the...   
...                                                 ...   
6101  But abnormal temperature spikes in February an...   

### Training

In [8]:

def pad_sen(input, max_len):
    if len(input) > max_len: return input[:max_len]
    return input+[PAD for _ in range(max_len-len(input))]

def saperate_evi(claim, evidence):
    ## inputs are tokens
    output = [CLS] + claim + [SEP] + evidence + [SEP]
    seg_li = [0 for _ in range(len(claim)+2)] + [1 for _ in range(len(evidence)+1)]
    return output, seg_li


class RelDataset(Dataset):
    def __init__(self, input_df, max_len_claim, max_len_evi, num_class):
        self.df = input_df
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.max_len_claim = max_len_claim
        self.max_len_evi = max_len_evi
        self.num_class = num_class
            
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        ## claim_text, claim_label, evidences
        #Selecting the sentence and label at the specified index in the data frame
        claim = self.df.loc[index, 'claim']
        label = self.df.loc[index, 'label']
        evidence = self.df.loc[index, 'evidence']

        #Preprocessing the claim
        claim = self.tokenizer.tokenize(claim)
        evidence = self.tokenizer.tokenize(evidence)
        claim = pad_sen(claim, self.max_len_claim)
        evidence = pad_sen(evidence, self.max_len_evi)
        input_token, seg_li = saperate_evi(claim, evidence)
        attn_mask = [1 if token != PAD else 0 for token in input_token]
        input_token = self.tokenizer.convert_tokens_to_ids(input_token)
        input_id = torch.tensor(input_token) #Converting the list to a pytorch tensor
        attn_mask = torch.tensor(attn_mask)
        seg_li = torch.tensor(seg_li)
        labels = np.zeros(self.num_class)
        np.put(labels, label, 1)
        label = torch.tensor(labels)
        
        # print(input_token, attn_mask, seg_li)
        
        return input_id, attn_mask, seg_li, label

class RelClassifier(nn.Module):
    def __init__(self, num_class, dropout = 0.1):
        super(RelClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = BertForSequenceClassification.from_pretrained('bert-base-uncased', problem_type="multi_label_classification")
        
        #Classification layer
        #input dimension is 768 because [CLS] embedding has a dimension of 768
        #output dimension is 1 because we're working with a binary classification problem
        self.linear = nn.Linear(768, num_class)
        self.dropout = nn.Dropout(dropout)
        self.num_class = num_class

    def forward(self, seq, attn_masks, seg_li):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''
        
        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert_layer(seq, attention_mask = attn_masks, token_type_ids = seg_li, output_hidden_states=True)
        cont_reps = outputs.hidden_states[-1][:,0]
        dropout_output = self.dropout(cont_reps)
        linear_output = self.linear(dropout_output)
        return linear_output
    
def get_accuracy(output, labels):
    return (output.argmax(dim=1) == labels.argmax(dim=1)).sum().item() / len(labels)

def encode_label(input, num_class):
    input = input.numpy()
    labels = []
    for i in range(len(input)):
            max_ind = np.argmax(input[i])
            cur_lab = np.zeros(num_class, dtype=int)
            np.put(cur_lab, max_ind, 1)
            labels.append(cur_lab)
    labels = torch.tensor(np.array(labels))
    return labels

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, seg_li, labels in dataloader:
            seq, attn_masks, seg_li, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_li.cuda(gpu), labels.cuda(gpu)
            output = net(seq, attn_masks, seg_li).cuda(gpu)
            mean_loss += criterion(output, labels)
            mean_acc += get_accuracy(output, labels)
            count += 1

    return mean_acc / count, mean_loss / count

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu, file_pre = "model_"):

    best_acc = 0
    best_loss = 99
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (seq, attn_masks, seg_li, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, seg_li, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_li.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model

            output = net(seq, attn_masks, seg_li)
            
            #Computing loss
            loss = criterion(output, labels)

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 30 == 0:
                acc = get_accuracy(output, labels)
                print("O: ", output[:3])
                print("L: ", labels[:3])
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc and dev_loss <= best_loss:
            print("Accuracy improved from {} to {}, Loss improved from {} to {}, saving model...".format(best_acc, dev_acc, best_loss, dev_loss))
            print("Saved: " + file_pre + 'model.dat')
            best_acc = dev_acc
            best_loss = dev_loss
            torch.save(net.state_dict(), file_pre + 'model.dat')
            # net.config.to_json_file(file_pre + '{}.ini'.format(ep))



In [9]:
TEST_FRA = 0.2
# train test splite
train_df, test_df = train_test_split(train_df, test_size = TEST_FRA, shuffle=True, random_state=RANDOM_SEED)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [10]:
PAD_CLAIM = 64
PAD_EVIDENCE = 64
BATCH_SIZE = 32
NO_WORKER = 4
TEST_FRA = 0.3
num_class = 2

# create data loader
train_ds = RelDataset(train_df[['claim', 'evidence', 'label']], PAD_CLAIM, PAD_EVIDENCE, num_class)
train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, num_workers = NO_WORKER)

test_ds = RelDataset(test_df[['claim', 'evidence', 'label']], PAD_CLAIM, PAD_EVIDENCE, num_class)
test_dl = DataLoader(test_ds, batch_size = BATCH_SIZE, num_workers = NO_WORKER)


In [11]:
# set up configuration
gpu = 0 #gpu ID
net = RelClassifier(num_class, dropout = 0.5)
net.cuda(gpu) #Enable gpu support for the model
criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
num_epoch = 3
# train the model
train(net, criterion, opti, train_dl, test_dl, num_epoch, gpu, file_pre = "rela_")


O:  tensor([[ 0.0030,  0.7104],
        [ 0.2904,  0.1179],
        [-0.4071, -0.0146]], device='cuda:0', grad_fn=<SliceBackward0>)
L:  tensor([[0., 1.],
        [0., 1.],
        [1., 0.]], device='cuda:0', dtype=torch.float64)
Iteration 0 of epoch 0 complete. Loss: 0.7279641309287399; Accuracy: 0.4375; Time taken (s): 10.820652723312378
O:  tensor([[-0.5852,  0.8893],
        [ 0.1548,  0.5235],
        [-1.1200,  1.0737]], device='cuda:0', grad_fn=<SliceBackward0>)
L:  tensor([[0., 1.],
        [0., 1.],
        [0., 1.]], device='cuda:0', dtype=torch.float64)
Iteration 30 of epoch 0 complete. Loss: 0.5797591452719644; Accuracy: 0.71875; Time taken (s): 15.72362232208252
O:  tensor([[-0.6275,  1.6055],
        [-0.7385,  1.5153],
        [-1.4453,  0.7481]], device='cuda:0', grad_fn=<SliceBackward0>)
L:  tensor([[0., 1.],
        [0., 1.],
        [1., 0.]], device='cuda:0', dtype=torch.float64)
Iteration 60 of epoch 0 complete. Loss: 0.5362702877027914; Accuracy: 0.8125; Time taken

### Evaluate

In [13]:
def predict_rel(net, dataloader, gpu):
    net.eval()
    pred = []
    with torch.no_grad():
        for seq, attn_masks, seg_li, _ in dataloader:
            seq, attn_masks, seg_li = seq.cuda(gpu), attn_masks.cuda(gpu), seg_li.cuda(gpu)
            ## HERE
            output = net(seq, attn_masks, seg_li)
            # output = np.array(output.tolist())
            # print(output.argmax())
            pred += torch.argmax(output, dim=1).tolist()

    return pred

gpu = 0 #gpu ID

pred_label = predict_rel(net, test_dl, gpu)


In [14]:
true_label = test_df.label.values.tolist()

print(pred_label)
print(true_label)
print(confusion_matrix(true_label, pred_label))
print(np.mean(np.array(true_label) == np.array(pred_label)))

[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 

: 